In [39]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [40]:
import yaml
with open('../config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [41]:

ENDPOINT = config["custom_vision"]["endpoint"];
training_key = config["custom_vision"]["training_key"]
prediction_key = config["custom_vision"]["prediction_key"]
prediction_resource_id = config["custom_vision"]["prediction_resource_id"]

project_id = config["project_id"]
if not config["project_id"]:
    raise ValueError

ValueError: 

## Download images and their tags

In [ ]:
from pathlib import Path

local_path = Path(f"../data/{project_id}")
Path.mkdir(local_path, exist_ok=True)
images_path = local_path / "Images"
Path.mkdir(images_path, exist_ok=True)

In [ ]:
import urllib
import http
import json

headers = {
 'Training-Key': training_key
}

params = urllib.parse.urlencode({
 # Format - int32. Maximum number of images to return. Defaults to 50, limited to 256.
 'take': 50,
 # Format - int32. Number of images to skip before beginning the image batch. Defaults to 0.
 'skip': 0,
 'orderBy': 'Oldest'
})

conn = http.client.HTTPSConnection(ENDPOINT)
conn.request(
 "GET", f"/customvision/v3.0/training/projects/{project_id}/images/tagged?%s" % params, "{body}", headers)
response = conn.getresponse()
data = response.read()
data_json = json.loads(data)

In [ ]:
import pandas as pd

df_cols = [
    "filename",
    "tag",
    "left",
    "top",
    "width",
    "height",
    "truncated",
    "difficult",
]
regions = []

for image in data_json:
    s_filename = image["id"] + ".png"

    # save image file
    try:
        urllib.request.urlretrieve(image["originalImageUri"], images_path / s_filename)
    except:
        print("Retry")
        urllib.request.urlretrieve(image["originalImageUri"], images_path / s_filename)

    # save tags
    for tag in image["regions"]:
        region = {"filename": s_filename}
        region["tag"] = tag["tagName"]
        region["truncated"] = 0
        region["difficult"] = 0

        region["left"] = tag["left"]
        region["top"] = tag["top"]
        region["width"] = tag["width"]
        region["height"] = tag["height"]
    
        regions.append(region)

conn.close()

df = pd.DataFrame(regions, columns=df_cols)
df.to_pickle(local_path / "object_detection.pkl")
df.head()


,filename,tag,left,top,width,height,truncated,difficult
0,28cb9cf9-be0e-498b-9b13-925f87263ef2.png,FireExtinguisher,0.468385,0.433470,0.040096,0.201993,0,0
1,dfb7ffc8-e1da-485e-9772-4c3ac1e59daa.png,FireExtinguisher,0.446044,0.403684,0.043340,0.247267,0,0
2,d376db8f-3843-4aee-9cca-074176ce416b.png,FireExtinguisher,0.637128,0.352135,0.062923,0.309699,0,0
3,13268293-3a85-48ad-9696-6efb94753c15.png,FireExtinguisher,0.698879,0.337952,0.082902,0.332119,0,0
4,40433882-d626-494d-9439-ef3d8402edad.png,FireExtinguisher,0.428180,0.487169,0.020824,0.081932,0,0
